In [1]:
import pandas as pd
from bertopic import BERTopic
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
import plotly.express as px

filter_relevant_features = True
shap_threshold = 0.001

bert_min_topic_size = 3

ImportError: cannot import name 'UMAP' from 'umap' (unknown location)

In [ ]:
df = pd.read_csv("feature_importance_shap_lgb.csv", sep=";")

In [ ]:
df = df[~df["description"].isna()].reset_index(drop=True)
# df["description"] = df["description"].fillna("NA")

In [ ]:
df.head()

In [ ]:
df["shap_value"].describe()

In [ ]:
df["shap_value"].hist(bins=20)

In [ ]:
if filter_relevant_features:
    df = df[df["shap_value"]>=shap_threshold].reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
stopwords = stopwords.words('english')
stopwords = stopwords+["et", "al", "http", "href", "omim", "org"]

In [ ]:
df["description_clean"] = df["description"].str.lower().str.replace('[^a-zA-Z]', ' ')
df["description_clean"] = df["description_clean"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [ ]:
docs = []
for case in df["description_clean"]:
    docs.append(case)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
topic_model = BERTopic(min_topic_size=bert_min_topic_size)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
len(probs)

In [ ]:
df["topic"] = topics

In [ ]:
df["topic"].value_counts()

In [ ]:
df_topic_model = topic_model.get_topic_info()

In [ ]:
topic_model.visualize_barchart(n_words=10, height=600)

In [ ]:
topic_model.get_representative_docs(0)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_term_rank()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

In [ ]:
hierarchical_topics

In [ ]:
print(topic_model.get_topic_tree(hierarchical_topics))

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
v = TfidfVectorizer()
x = v.fit_transform(df['description_clean'])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
tf_idf_words = df1.columns
df = pd.concat([df, df1], axis=1)

scaler = StandardScaler()
scaler_fit = scaler.fit(df1.values) 
df_scaled = scaler_fit.transform(df1.values)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = tf_idf_words

reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_scaled)
embedding.shape

df["UMAP_X"] = embedding[:, 0]
df["UMAP_Y"] = embedding[:, 1]

df["topic_bert"] = df["topic"].astype(int).astype(str)

In [ ]:
fig = px.scatter(df, x="UMAP_X", y="UMAP_Y", color="topic_bert"#, hover_data=['feature']
                 )
fig.show()